<a href="https://colab.research.google.com/github/sparks-baird/CrabNet/blob/main/examples/colab/RoosterCrab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -UI mat_discover matbench # for CrabNet/ElMD and benchmarking

  Using cached mat_discover-2.1.4-py2.py3-none-any.whl (37 kB)
  Using cached matbench-0.5-py3-none-any.whl (9.9 MB)
  Using cached chem_wasserstein-1.0.9-py3-none-any.whl (1.0 MB)
  Using cached plotly-5.6.0-py2.py3-none-any.whl (27.7 MB)
  Using cached scipy-1.7.3-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (38.1 MB)
  Using cached numba-0.55.1-1-cp37-cp37m-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.3 MB)
  Using cached scikit_learn-1.0.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.8 MB)
  Using cached seaborn-0.11.2-py3-none-any.whl (292 kB)
  Using cached hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl
  Using cached crabnet-2.0.3-py3-none-any.whl (34.6 MB)
  Using cached kaleido-0.2.1-py2.py3-none-manylinux1_x86_64.whl (79.9 MB)
  Using cached matplotlib-3.5.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Using cached umap_learn-0.5.

In [4]:
%%capture
from torch import __version__ as TORCH_VERSION
!pip install pymatgen pybtex retrying  # install requirements to query sample data
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{TORCH_VERSION}.html  # install torch scatter for aviary
!pip install -U git+https://github.com/CompRhys/aviary.git  # install aviary

In [6]:
import os 
import numpy as np
import torch

from aviary.utils import results_multitask, train_ensemble

In [7]:
resume = False
fine_tune = None
transfer = None

optim = "AdamW"
learning_rate = 3e-4
momentum = 0.9
weight_decay = 1e-6
batch_size = 128
workers = 0  
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

targets = ["volume_per_atom"]
tasks = ["regression"]
losses = ["L1"]
robust = True

data_seed = 42
test_size = 0.2
sample = 1

ensemble = 1
run_id = 1
epochs = 100
log = False

# NOTE setting workers to zero means that the data is loaded in the main
# process and enables caching

data_params = {
    "batch_size": batch_size,
    "num_workers": workers,
    "pin_memory": False,
    "shuffle": True,
}

setup_params = {
    "optim": optim,
    "learning_rate": learning_rate,
    "weight_decay": weight_decay,
    "momentum": momentum,
    "device": device,
}

restart_params = {
    "resume": resume,
    "fine_tune": fine_tune,
    "transfer": transfer,
}

task_dict = dict(zip(targets, tasks))
loss_dict = dict(zip(targets, losses))

In [ ]:
from aviary.roost.data import CompositionData, collate_batch as roost_cb
from aviary.roost.model import Roost
from matbench import MatbenchBenchmark

torch.manual_seed(0)  # ensure reproducible results

elem_emb = "matscholar200"
model_name = "roost-reg-test"

data_params["collate_fn"] = roost_cb
data_params["shuffle"] = True 

mb = MatbenchBenchmark(subsets=["matbench_expt_gap"]

dataset = CompositionData(
    df=df,
    elem_emb=elem_emb,
    task_dict=task_dict, 
)
n_targets = dataset.n_targets
elem_emb_len = dataset.elem_emb_len

train_idx = list(range(len(dataset)))

print(f"using {test_size} of training set as test set")
train_idx, test_idx = split(train_idx, random_state=data_seed, test_size=test_size)
test_set = torch.utils.data.Subset(dataset, test_idx)

print("No validation set used, using test set for evaluation purposes")
# NOTE that when using this option care must be taken not to
# peak at the test-set. The only valid model to use is the one
# obtained after the final epoch where the epoch count is
# decided in advance of the experiment.
val_set = test_set

train_set = torch.utils.data.Subset(dataset, train_idx[0::sample])

model_params = {
    "task_dict": task_dict,
    "robust": robust,
    "n_targets": n_targets,
    "elem_emb_len": elem_emb_len,
    "elem_fea_len": 64,
    "n_graph": 3,
    "elem_heads": 3,
    "elem_gate": [256],
    "elem_msg": [256],
    "cry_heads": 3,
    "cry_gate": [256],
    "cry_msg": [256],
    "trunk_hidden": [128, 128],
    "out_hidden": [64, 64],
}

os.makedirs(f"models/{model_name}", exist_ok=True)
os.makedirs(f"results/{model_name}", exist_ok=True)

train_ensemble(
    model_class=Roost,
    model_name=model_name,
    run_id=run_id,
    ensemble_folds=ensemble,
    epochs=epochs,
    train_set=train_set,
    val_set=val_set,
    log=log,
    data_params=data_params,
    setup_params=setup_params,
    restart_params=restart_params,
    model_params=model_params,
    loss_dict=loss_dict,
)

data_params["shuffle"] = False  # need fixed data order due to ensembling

roost_results_dict = results_multitask(
    model_class=Roost,
    model_name=model_name,
    run_id=run_id,
    ensemble_folds=ensemble,
    test_set=test_set,
    data_params=data_params,
    robust=robust,
    task_dict=task_dict,
    device=device,
    eval_type="checkpoint",
    save_results=False,
)


using 0.2 of training set as test set
No validation set used, using test set for evaluation purposes
Total Number of Trainable Parameters: 952,977
Dummy MAE: 4.4372
Getting Validation Baseline


  0%|          | 0/1 [00:00<?, ?it/s]

Validation Baseline - volume_per_atom: MAE 4.883


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: [1/100]
Train 		: volume_per_atom - Loss 0.951	MAE 4.253	RMSE 6.333	
Validation 	: volume_per_atom - Loss 1.090	MAE 4.854	RMSE 10.499	


...

Epoch: [98/100]
Train 		: volume_per_atom - Loss -0.104	MAE 2.037	RMSE 4.170	
Validation 	: volume_per_atom - Loss 0.846	MAE 3.251	RMSE 9.009	


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: [99/100]
Train 		: volume_per_atom - Loss -0.117	MAE 2.010	RMSE 4.112	
Validation 	: volume_per_atom - Loss 1.005	MAE 3.183	RMSE 9.014	


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: [100/100]
Train 		: volume_per_atom - Loss -0.154	MAE 1.977	RMSE 4.131	
Validation 	: volume_per_atom - Loss 0.913	MAE 3.160	RMSE 8.954	
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
------------Evaluate model on Test Set------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Evaluating Model

Task: 'volume_per_atom' on Test Set
Model Performance Metrics:
R2 Score: 0.2393 
MAE: 3.1598
RMSE: 8.9544
